In [3]:
import torch
import torch.nn as nn
from torch.export import export

In [4]:
class M(torch.nn.Module):
    def __init__(self, d_in, d_out):
        super(M, self).__init__()
        self.layer = nn.Linear(d_in, d_out)
    def forward(self, x):
        return self.layer(x)

arg = torch.randn(2, 10)
model = M(10, 20)

exported = export(model, args=(arg,))
print(exported)

ExportedProgram:
    class GraphModule(torch.nn.Module):
        def forward(self, arg0_1: f32[20, 10], arg1_1: f32[20], arg2_1: f32[2, 10]):
            # 
            permute: f32[10, 20] = torch.ops.aten.permute.default(arg0_1, [1, 0]);  arg0_1 = None
            addmm: f32[2, 20] = torch.ops.aten.addmm.default(arg1_1, arg2_1, permute);  arg1_1 = arg2_1 = permute = None
            return (addmm,)
            
Graph Signature: ExportGraphSignature(parameters=['L__self___layer.weight', 'L__self___layer.bias'], buffers=[], user_inputs=['arg2_1'], user_outputs=['addmm'], inputs_to_parameters={'arg0_1': 'L__self___layer.weight', 'arg1_1': 'L__self___layer.bias'}, inputs_to_buffers={}, buffers_to_mutate={}, backward_signature=None, assertion_dep_token=None)
Symbol to range: {}



In [6]:
for n in exported.graph.nodes:
    print(n)

arg0_1
arg1_1
arg2_1
permute
addmm
output


In [17]:
for n in exported.graph.nodes:
    print(n, n.all_input_nodes)

arg0_1 []
arg1_1 []
arg2_1 []
permute [arg0_1]
addmm [arg1_1, arg2_1, permute]
output [addmm]


In [21]:
for n in exported.graph.nodes:
    print(n, n.all_input_nodes)
    for a in n.all_input_nodes:
        print('fake tensor', a, a.meta['val'])

arg0_1 []
arg1_1 []
arg2_1 []
permute [arg0_1]
fake tensor arg0_1 FakeTensor(..., size=(20, 10))
addmm [arg1_1, arg2_1, permute]
fake tensor arg1_1 FakeTensor(..., size=(20,))
fake tensor arg2_1 FakeTensor(..., size=(2, 10))
fake tensor permute FakeTensor(..., size=(10, 20))
output [addmm]
fake tensor addmm FakeTensor(..., size=(2, 20))


In [22]:
def count_addmm_flops(node):
    # ignore bias
    t1 = node.all_input_nodes[1].meta['val']
    t2 = node.all_input_nodes[2].meta['val']
    m, k = t1.size()
    n = t2.size(1)
    return 2 * m * k * n

nodes = list(exported.graph.nodes)
addmm_node = nodes[-2]
print(addmm_node)

print('addmm flops', count_addmm_flops(addmm_node))


addmm
addmm flops 800


In [28]:
addmm_node.target.name()

'aten::addmm'

In [29]:
total_flops = 0
for n in exported.graph.nodes:
    if isinstance(n.target, torch._ops.OpOverload):
        if n.target.name() == 'aten::addmm':
            total_flops += count_addmm_flops(n)
print('total flops', total_flops)


total flops 800


Now let's implement the over nodes

In [30]:
def count_bmm_flops(node):
    t1 = node.all_input_nodes[0].meta['val']
    t2 = node.all_input_nodes[1].meta['val']
    batch_size, m, k = t1.size()
    n = t2.size(2)
    return 2 * batch_size * m * k * n

def count_mm_flops(node):
    t1 = node.all_input_nodes[0].meta['val']
    t2 = node.all_input_nodes[1].meta['val']
    m, k = t1.size()
    n = t2.size(1)
    return 2 * m * k * n

total_flops = 0
for n in exported.graph.nodes:
    if isinstance(n.target, torch._ops.OpOverload):
        if n.target.name() == 'aten::addmm':
            total_flops += count_addmm_flops(n)
        elif n.target.name() == 'aten::mm':
            total_flops += count_mm_flops(n)
        elif n.target.name() == 'aten::bmm':
            total_flops += count_bmm_flops(n)
print('total flops', total_flops)


total flops 800


Let's adjust the model so that we have an example for all 3 instructions.

In [50]:
class M(torch.nn.Module):
    def __init__(self):
        super(M, self).__init__()
        self.layer = nn.Linear(10, 20)
        self.param = nn.Parameter(torch.zeros(20, 20))
        self.param2 = nn.Parameter(torch.zeros(2, 10, 2))
    def forward(self, x):
        x = self.layer(x)
        x = x @ self.param
        x = x.view(2, 2, 10)
        x = x @ self.param2
        return x.view(2, -1)

In [52]:
arg = torch.randn(2, 10)
model = M()

exported = export(model, args=(arg,))
print(exported)

ExportedProgram:
    class GraphModule(torch.nn.Module):
        def forward(self, arg0_1: f32[20, 20], arg1_1: f32[2, 10, 2], arg2_1: f32[20, 10], arg3_1: f32[20], arg4_1: f32[2, 10]):
            # 
            permute: f32[10, 20] = torch.ops.aten.permute.default(arg2_1, [1, 0]);  arg2_1 = None
            addmm: f32[2, 20] = torch.ops.aten.addmm.default(arg3_1, arg4_1, permute);  arg3_1 = arg4_1 = permute = None
            mm: f32[2, 20] = torch.ops.aten.mm.default(addmm, arg0_1);  addmm = arg0_1 = None
            view: f32[2, 2, 10] = torch.ops.aten.view.default(mm, [2, 2, 10]);  mm = None
            expand: f32[2, 2, 10] = torch.ops.aten.expand.default(view, [2, 2, 10]);  view = None
            view_1: f32[2, 2, 10] = torch.ops.aten.view.default(expand, [2, 2, 10]);  expand = None
            expand_1: f32[2, 10, 2] = torch.ops.aten.expand.default(arg1_1, [2, 10, 2]);  arg1_1 = None
            view_2: f32[2, 10, 2] = torch.ops.aten.view.default(expand_1, [2, 10, 2]);  expand

In [53]:
total_flops = 0
for n in exported.graph.nodes:
    if isinstance(n.target, torch._ops.OpOverload):
        if n.target.name() == 'aten::addmm':
            total_flops += count_addmm_flops(n)
        elif n.target.name() == 'aten::mm':
            total_flops += count_mm_flops(n)
        elif n.target.name() == 'aten::bmm':
            total_flops += count_bmm_flops(n)
print('total flops', total_flops)

total flops 2560


Let's make a function for this.

In [58]:
from collections import defaultdict

def flops_report(exported_program):
    flops = 0
    flops_by_type = defaultdict(int)  # Store FLOPs for each type of operation

    print('Counting FLOPs for the exported graph...')
    
    op_to_count_func = {
        "aten::bmm":count_bmm_flops,
        "aten::addmm":count_addmm_flops,
        "aten::mm":count_mm_flops,
    }

    print(f"Number of nodes in the exported graph: {len(exported_program.graph.nodes)}")
    print(f"Tracking operations {list(op_to_count_func.keys())}")

    all_ops = set()

    for n in exported_program.graph.nodes:
        if isinstance(n.target, torch._ops.OpOverload):
            op_type = n.target.name()
            all_ops.add(op_type)
            flops_for_node = 0

            if op_type in op_to_count_func:
                flops_for_node = op_to_count_func[op_type](n)
                flops_by_type[op_type] += flops_for_node
                flops += flops_for_node

    print(f"All operations seen: {list(all_ops)}")
    
    print("\nFLOPs by Operation Type:")
    for op_type, flops_for_type in flops_by_type.items():
        percentage = (flops_for_type / flops) * 100 if flops != 0 else 0
        print(f"{op_type}: {flops_for_type} FLOPs ({percentage:.4f}%)")
    print(f"\nTotal FLOPs: {flops}")
    
    return flops

In [59]:
flops_report(exported)

Counting FLOPs for the exported graph...
Number of nodes in the exported graph: 17
Tracking operations ['aten::bmm', 'aten::addmm', 'aten::mm']
All operations seen: ['aten::bmm', 'aten::addmm', 'aten::view', 'aten::permute', 'aten::expand', 'aten::mm']

FLOPs by Operation Type:
aten::addmm: 800 FLOPs (31.2500%)
aten::mm: 1600 FLOPs (62.5000%)
aten::bmm: 160 FLOPs (6.2500%)

Total FLOPs: 2560


2560

In [82]:
"""
Full definition of a GPT Language Model, all of it in this single file.
References:
1) the official GPT-2 TensorFlow implementation released by OpenAI:
https://github.com/openai/gpt-2/blob/master/src/model.py
2) huggingface/transformers PyTorch implementation:
https://github.com/huggingface/transformers/blob/main/src/transformers/models/gpt2/modeling_gpt2.py
"""

import math
import inspect
from dataclasses import dataclass

import torch
import torch.nn as nn
from torch.nn import functional as F

class LayerNorm(nn.Module):
    """ LayerNorm but with an optional bias. PyTorch doesn't support simply bias=False """

    def __init__(self, ndim, bias):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(ndim))
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None

    def forward(self, input):
        return F.layer_norm(input, self.weight.shape, self.weight, self.bias, 1e-5)

class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=config.bias)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
        # regularization
        self.attn_dropout = nn.Dropout(config.dropout)
        self.resid_dropout = nn.Dropout(config.dropout)
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.dropout = config.dropout
        # flash attention make GPU go brrrrr but support is only in PyTorch >= 2.0
        # self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')
        self.flash = False
        if not self.flash:
            print("WARNING: using slow attention. Flash Attention requires PyTorch >= 2.0")
            # causal mask to ensure that attention is only applied to the left in the input sequence
            self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                        .view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        q, k, v  = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        if self.flash:
            # efficient attention using Flash Attention CUDA kernels
            y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)
        else:
            # manual implementation of attention
            att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
            att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
            att = F.softmax(att, dim=-1)
            att = self.attn_dropout(att)
            y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.c_proj(y))
        return y

class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd, bias=config.bias)
        self.gelu    = nn.GELU()
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        x = self.dropout(x)
        return x

class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = LayerNorm(config.n_embd, bias=config.bias)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = LayerNorm(config.n_embd, bias=config.bias)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

@dataclass
class GPTConfig:
    block_size: int = 1024
    vocab_size: int = 50304 # GPT-2 vocab_size of 50257, padded up to nearest multiple of 64 for efficiency
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768
    dropout: float = 0.0
    bias: bool = True # True: bias in Linears and LayerNorms, like GPT-2. False: a bit better and faster

class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.vocab_size is not None
        assert config.block_size is not None
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            drop = nn.Dropout(config.dropout),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = LayerNorm(config.n_embd, bias=config.bias),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        # with weight tying when using torch.compile() some warnings get generated:
        # "UserWarning: functional_call was passed multiple values for tied weights.
        # This behavior is deprecated and will be an error in future versions"
        # not 100% sure what this is, so far seems to be harmless. TODO investigate
        self.transformer.wte.weight = self.lm_head.weight # https://paperswithcode.com/method/weight-tying

        # init all weights
        self.apply(self._init_weights)
        # apply special scaled init to the residual projections, per GPT-2 paper
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * config.n_layer))

        # report number of parameters
        print("number of parameters: %.2fM" % (self.get_num_params()/1e6,))

    def get_num_params(self, non_embedding=True):
        """
        Return the number of parameters in the model.
        For non-embedding count (default), the position embeddings get subtracted.
        The token embeddings would too, except due to the parameter sharing these
        params are actually used as weights in the final layer, so we include them.
        """
        n_params = sum(p.numel() for p in self.parameters())
        if non_embedding:
            n_params -= self.transformer.wpe.weight.numel()
        return n_params

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.config.block_size, f"Cannot forward sequence of length {t}, block size is only {self.config.block_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device) # shape (t)

        # forward the GPT model itself
        tok_emb = self.transformer.wte(idx) # token embeddings of shape (b, t, n_embd)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (t, n_embd)
        x = self.transformer.drop(tok_emb + pos_emb)
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        if targets is not None:
            # if we are given some desired targets also calculate the loss
            logits = self.lm_head(x)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        else:
            # inference-time mini-optimization: only forward the lm_head on the very last position
            logits = self.lm_head(x[:, [-1], :]) # note: using list [-1] to preserve the time dim
            loss = None

        return logits

In [83]:
config = GPTConfig()
print(config)

GPTConfig(block_size=1024, vocab_size=50304, n_layer=12, n_head=12, n_embd=768, dropout=0.0, bias=True)


In [88]:
gpt = GPT(config)
print(gpt)

number of parameters: 123.69M
GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50304, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=768, out_features=50304, bias=False)
)


We made two adjustments:

1. `self.flash` is always False. Otherwise it will use the flash attention instruction `aten::_scaled_dot_product_flash_attention` which we are not currently tracking.
2. `forward(..)` only returns logits and not loss. `export` doesn't like multiple outputs.

In [85]:
gpt_input = torch.zeros((1, config.block_size), dtype=torch.long)
print(gpt_input)
print(gpt_input.size())

tensor([[0, 0, 0,  ..., 0, 0, 0]])
torch.Size([1, 1024])


In [89]:
exported = export(gpt, args=(gpt_input,))
print(exported)

ExportedProgram:
    class GraphModule(torch.nn.Module):
        def forward(self, arg0_1: f32[768], arg1_1: f32[768], arg2_1: f32[768], arg3_1: f32[768], arg4_1: f32[768], arg5_1: f32[768], arg6_1: f32[768], arg7_1: f32[768], arg8_1: f32[768], arg9_1: f32[768], arg10_1: f32[768], arg11_1: f32[768], arg12_1: f32[768], arg13_1: f32[768], arg14_1: f32[768], arg15_1: f32[768], arg16_1: f32[768], arg17_1: f32[768], arg18_1: f32[768], arg19_1: f32[768], arg20_1: f32[768], arg21_1: f32[768], arg22_1: f32[768], arg23_1: f32[768], arg24_1: f32[768], arg25_1: f32[768], arg26_1: f32[768], arg27_1: f32[768], arg28_1: f32[768], arg29_1: f32[768], arg30_1: f32[768], arg31_1: f32[768], arg32_1: f32[768], arg33_1: f32[768], arg34_1: f32[768], arg35_1: f32[768], arg36_1: f32[768], arg37_1: f32[768], arg38_1: f32[768], arg39_1: f32[768], arg40_1: f32[768], arg41_1: f32[768], arg42_1: f32[768], arg43_1: f32[768], arg44_1: f32[768], arg45_1: f32[768], arg46_1: f32[768], arg47_1: f32[768], arg48_1: f32[76

In [87]:
flops_report(exported)

Counting FLOPs for the exported graph...
Number of nodes in the exported graph: 887
Tracking operations ['aten::bmm', 'aten::addmm', 'aten::mm']
All operations seen: ['aten::add.Tensor', 'aten::addmm', 'aten::where.self', 'aten::view', 'aten::permute', 'aten::lift_fresh_copy', 'aten::embedding', 'aten::clone', 'aten::bmm', 'aten::split.Tensor', 'aten::mul.Tensor', 'aten::slice.Tensor', 'aten::mm', 'aten::eq.Scalar', 'aten::gelu', 'aten::_softmax', 'aten::arange.start_step', 'aten::expand', 'aten::scalar_tensor', 'aten::index.Tensor', 'aten::native_layer_norm']

FLOPs by Operation Type:
aten::addmm: 173946175488 FLOPs (81.7885%)
aten::bmm: 38654705664 FLOPs (18.1752%)
aten::mm: 77266944 FLOPs (0.0363%)

Total FLOPs: 212678148096


212678148096

Delve into the FLOPs for a Block.

In [92]:
b = Block(config)

In [93]:
arg = torch.randn(1, 1024, 768)

In [95]:
flops_report(export(b, args=(arg,)))

Counting FLOPs for the exported graph...
Number of nodes in the exported graph: 74
Tracking operations ['aten::bmm', 'aten::addmm', 'aten::mm']
All operations seen: ['aten::bmm', 'aten::eq.Scalar', 'aten::add.Tensor', 'aten::_softmax', 'aten::gelu', 'aten::split.Tensor', 'aten::addmm', 'aten::where.self', 'aten::view', 'aten::permute', 'aten::expand', 'aten::mul.Tensor', 'aten::scalar_tensor', 'aten::slice.Tensor', 'aten::clone', 'aten::native_layer_norm']

FLOPs by Operation Type:
aten::addmm: 14495514624 FLOPs (81.8182%)
aten::bmm: 3221225472 FLOPs (18.1818%)

Total FLOPs: 17716740096


17716740096

FLOPs for LM head. Shapes taken from graph. 1 is there because in inference we only care about the final time dimension.

f32[1, 768]
f32[768, 50304]
2 * 1 * 768 * 50304

In [99]:
(17716740096 * 12) + (2 * 1 * 768 * 50304)

212678148096

In [100]:
(17716740096 * 12) + (2 * 1024 * 768 * 50304) # for training (1024 is entire sequence)

291722231808

In [101]:
2 * 1 * 768 * 50304

77266944

In [102]:
abs(212678148096 - 212678148096)


0